In [ ]:
import json
import numpy as np
import seismic
from seismic import SeismicDatasetLV, SeismicIndexLV
from tqdm import tqdm

In [ ]:

document_path = ""



dataset = SeismicDatasetLV()
string_type  = seismic.get_seismic_string()

with open(document_path, "r") as file:
    for line in tqdm(file):
        line_data = json.loads(line.strip())
        vs = np.array([v for v in line_data["vector"].values()], dtype=np.float32)
        ks = np.array([k for k in line_data["vector"].keys()], dtype=string_type)
        dataset.add_document(str(line_data["id"]), ks, vs)
        

In [ ]:
index = SeismicIndexLV.build_from_dataset(dataset)

In [ ]:
import numpy as np
import json

file_path = ""

queries = []
with open(file_path, 'r') as f:
    for line in f:
        queries.append(json.loads(line))

MAX_TOKEN_LEN = 30
string_type  = f'U{MAX_TOKEN_LEN}'

queries_ids = np.array([q['id'] for q in queries], dtype=string_type)

query_components = []
query_values = []

for query in queries[:1000]:
    vector = query['vector']
    query_components.append(np.array(list(vector.keys()), dtype=string_type))
    query_values.append(np.array(list(vector.values()), dtype=np.float32))

In [ ]:
results = index.batch_search(
    queries_ids=queries_ids,
    query_components=query_components,
    query_values=query_values,
    k=10,
    query_cut=20,
    heap_factor=0.7,
    n_knn=0,
    sorted=True, #specified even if default value
    num_threads=1,
)

In [ ]:
import ir_measures
import ir_datasets

# add your ir_dataset dataset string id below, e.g., "beir/quora/test"
ir_dataset_string = "beir/nfcorpus/test"

ir_results = [ir_measures.ScoredDoc(query_id, doc_id, score) for r in results for (query_id, score, doc_id) in r]
qrels = ir_datasets.load(ir_dataset_string).qrels

In [ ]:
from ir_measures import *
metric_name = "nDCG@10" # on BEIR/quora
ir_measure = ir_measures.parse_measure(metric_name)

qrels = ir_datasets.load(ir_dataset_string).qrels
ir_measures.calc_aggregate([ir_measure], qrels, ir_results)